In [1]:
import igraph, pandas as pd, numpy as np, plotly.graph_objects as go
from igraph import Graph, EdgeSeq

# Set up tree with "igraph"

In [2]:
feeders_2D = pd.read_csv("../data/bookings_ids+pax_date.csv")
print('feeders_2D.shape =', feeders_2D.shape, '.')

# destinations_1D = feeders_2D['od_nf'].copy()
# destinations_1D = destinations_1D.str.slice(start = 3, stop = 6, step = 1)
# destinations_unique_1D = np.array(destinations_1D.unique())
# destinations_unique_1D = np.sort(destinations_unique_1D)

flight_ID_1D = feeders_2D['id_nf'].copy()
flight_ID_unique_1D = np.array(flight_ID_1D.unique())

feeders_2D.shape = (4124, 18) .


In [3]:
FSU_2D = pd.read_csv("../data/FSU_fully_cleaned.csv")

print('FSU_2D.shape =', FSU_2D.shape, '.')

FSU_2D.shape = (49862, 76) .


In [4]:
origin = "PTY"

# def flight_data_conversion(sch_dep_year_Z, sch_dep_month_Z, sch_dep_day_Z, origin, destination, sch_dep_hour_Z, sch_dep_minute_Z, flight_number):
    
#     print('Scheduled departure date (Zulu): \t year =', sch_dep_year_Z, ', \t month =', sch_dep_month_Z, ', \t day =', sch_dep_day_Z, '.')
#     print('Origin =', origin, ', \t destination =', destination, '.')
#     print('Scheduled departure time (Zulu): \t hour =', sch_dep_hour_Z, ', \t minute =', sch_dep_minute_Z, '.')
#     print('Flight number =', flight_number, '.')

#     month_conversion = { "January": "01",
#                         "February" : "02",
#                         "March" : "03",
#                         "April" : "04",
#                         "May" : "05",
#                         "June" : "06",
#                         "July" : "07",
#                         "August" : "08",
#                         "September" : "09",
#                         "October" : "10",
#                         "November" : "11",
#                         "December" : "12" }      

#     month_number_string = month_conversion[sch_dep_month_Z]

#     if day.value < 10:
#         day_number_string = "0" + str(sch_dep_day_Z)
#     else:
#         day_number_string = str(sch_dep_day_Z)

#     sch_dep_date_Z = str(sch_dep_year_Z) + "-" + month_number_string + "-" + day_number_string
#     OD = origin + destination
#     sch_dep_time_Z = str(sch_dep_hour_Z) + ":" + str(sch_dep_minute_Z)
#     flight_unique_ID = str(sch_dep_year_Z) + "/" + month_number_string + "/" + day_number_string + origin + destination + str(sch_dep_hour_Z) + ":" + str(sch_dep_minute_Z) + str(flight_number)
#     print('flight_unique_ID =', flight_unique_ID, '.')

#     return sch_dep_date_Z, OD, sch_dep_time_Z, flight_unique_ID

In [5]:
# def get_feeder_flights(outbound_1_flight_ID): # outbound_1_dep_dtz, outbound_1_OD, outbound_1_dep_tmz, outbound_1_flt_num):
    
    # return_flight_1D = FSU_2D.loc[0, :]
    # print('\n return_flight_1D =', return_flight_1D, '. \n')

    # outbound_1_flight_ID = sch_dep_date_Z + OD + sch_dep_time_Z + str(outbound_1_flt_num)
    # outbound_1_flight_ID = outbound_1_flight_ID.replace("-", "/")
    
    # feeder_flights_2D = feeders_2D[feeders_2D['id_nf'] == outbound_1_flight_ID]
    # n_feeders = feeder_flights_2D.shape[0]
    
    # print('The outbound flight', outbound_1_flight_ID, 'has the following', n_feeders, 'inbound flights as feeders: \n')
    # print(feeder_flights_2D[['id_f', 'pax_f']])
    
    # return feeder_flights_2D

In [6]:
def get_tooltips_1 (outbound_1_flight_ID):

#     print('Selected outbound flight details:')
#     print('\t Scheduled departure date (Zulu):', outbound_1_flight_ID[0:10], '.')
#     print('\t Origin:', outbound_1_flight_ID[10:13], '. \t\t Destination:', outbound_1_flight_ID[13:16], '.')
#     print('\t Scheduled departure time (Zulu):', outbound_1_flight_ID[16:21], '.')
#     print('\t Flight number:', outbound_1_flight_ID[21:24], '. \n')

    feeder_flights_2D = feeders_2D[feeders_2D['id_nf'] == outbound_1_flight_ID]
    
    outbound_1_dep_dtz = feeder_flights_2D['dep_dtz_nf'].iloc[0]
    outbound_1_OD = feeder_flights_2D['od_nf'].iloc[0]
    outbound_1_dep_tmz = feeder_flights_2D['dep_tmz_nf'].iloc[0]
    outbound_1_flt_num = feeder_flights_2D['flt_num_nf'].iloc[0]
    n_feeders = feeder_flights_2D.shape[0]
    pax_transfer_1_PTY_1D = feeder_flights_2D['pax_f']
    
    # Hover information for nodes (flights):
    arr_delay_1D = []
    dep_delay_1D =  []
    flt_num_1D = []
    flt_duration_1D = []
    bank_code_1D = []
    OD_1D = []
    tail_1D = []
    dep_date_Z_1D = []
    dep_time_Z_1D = []
    arr_date_Z_1D = []
    arr_time_Z_1D = []
    outbound_1_SCH_ARR_DTMZ = -10000
    
    for x in range(n_feeders):
        d = FSU_2D[(FSU_2D['SCH_DEP_DTZ'] == feeder_flights_2D['dep_dtz_f'].iloc[x]) & (FSU_2D['OD'] == feeder_flights_2D['od_f'].iloc[x]) & (FSU_2D['SCH_DEP_TMZ'] == feeder_flights_2D['dep_tmz_f'].iloc[x]) & (FSU_2D['FLT_NUM'] == feeder_flights_2D['flt_num_f'].iloc[x])]
        if d.shape[0] == 1:
            arr_delay_1D.extend(d['ARR_DELAY_MINUTES'])
            dep_delay_1D.extend(d['DEP_DELAY_MINUTES'])
            flt_num_1D.extend(d['FLT_NUM'])
            flt_duration_1D.extend(d['FLT_ACTUAL_HR'])
            OD_1D.extend(d['OD'])
            tail_1D.extend(d['TAIL'])
            bank_code_1D.extend(d['BANK_CD'].values)
            dep_date_Z_1D.extend(d['SCH_DEP_DTZ'].values)
            dep_time_Z_1D.extend(d['SCH_DEP_TMZ'].values)
            arr_date_Z_1D.extend(d['SCH_ARR_DTZ'].values)
            arr_time_Z_1D.extend(d['SCH_ARR_TMZ'].values)
        else:
            arr_delay_1D.append(-10000)
            dep_delay_1D.append(-10000)
            flt_duration_1D.append(-10000)
            flt_num_1D.append(-10000)
            OD_1D.extend('0')
            tail_1D.extend('0')
            bank_code_1D.extend('0')
            dep_date_Z_1D.extend('0')
            dep_time_Z_1D.extend('0')
            arr_date_Z_1D.extend('0')
            arr_time_Z_1D.extend('0')
            print('Error: Feeder data not found.')

    D = FSU_2D[(FSU_2D['SCH_DEP_DTZ'] == outbound_1_dep_dtz) & (FSU_2D['OD'] == outbound_1_OD) & (FSU_2D['SCH_DEP_TMZ'] == outbound_1_dep_tmz) & (FSU_2D['FLT_NUM'] == outbound_1_flt_num)]
    # print('D.shape =', D.shape, '. \n')

    if D.shape[0] > 0:
        arr_delay_1D.extend(D['ARR_DELAY_MINUTES'])
        dep_delay_1D.extend(D['DEP_DELAY_MINUTES'])
        flt_num_1D.extend(D['FLT_NUM'])
        flt_duration_1D.extend(D['FLT_ACTUAL_HR'])
        bank_code_1D.extend(D['BANK_CD'])
        OD_1D.extend(D['OD'])
        tail_1D.extend(D['TAIL'])
        dep_date_Z_1D.extend(D['SCH_DEP_DTZ'])
        dep_time_Z_1D.extend(D['SCH_DEP_TMZ'])
        arr_date_Z_1D.extend(D['SCH_ARR_DTZ'])
        arr_time_Z_1D.extend(D['SCH_ARR_TMZ'])
        outbound_1_SCH_ARR_DTMZ = D['SCH_ARR_DTMZ'].values
        # print('outbound_1_SCH_ARR_DTMZ =', outbound_1_SCH_ARR_DTMZ, '. \n')
    else:
        arr_delay_1D.append(0)
        dep_delay_1D.append(0)
        flt_num_1D.append(0)
        flt_duration_1D.append(0)
        OD_1D.append('0')
        bank_code_1D.append('0')
        tail_1D.append('0')
        dep_date_Z_1D.append('0')
        dep_time_Z_1D.append('0')
        arr_date_Z_1D.append('0')
        arr_time_Z_1D.append('0')
        print('\n Error: Outbound flight \"' + outbound_1_flight_ID + '\" is not found in file \"FSU_fully_cleaned.csv\". \n')

    return n_feeders, OD_1D, tail_1D, dep_delay_1D, arr_delay_1D, flt_num_1D, flt_duration_1D, bank_code_1D, dep_date_Z_1D, dep_time_Z_1D, arr_date_Z_1D, arr_time_Z_1D, outbound_1_SCH_ARR_DTMZ

In [7]:
def get_tooltips_2 (outbound_1_flight_ID):
    
    n_feeders, OD_1D, tail_1D, dep_delay_1D, arr_delay_1D, flt_num_1D, flt_duration_1D, bank_code_1D, dep_date_Z_1D, dep_time_Z_1D, arr_date_Z_1D, arr_time_Z_1D, outbound_1_SCH_ARR_DTMZ = get_tooltips_1 (outbound_1_flight_ID)

    outbound_1_OD = OD_1D[len(OD_1D) - 1]
    tail_1D.extend(tail_1D[n_feeders :])
    reverse_OD = outbound_1_OD[3:6] + outbound_1_OD[0:3]     # OD of return flight is reverse of OD of outbound flight 1.
    OD_1D.append(reverse_OD)

    return_flight_1D = []
    
    if tail_1D[n_feeders + 1] != '0':
        return_flights_2D = FSU_2D[(FSU_2D['TAIL'] == tail_1D[n_feeders+1]) & (FSU_2D['OD'] == reverse_OD) & (FSU_2D['SCH_DEP_DTMZ'] > outbound_1_SCH_ARR_DTMZ[0])]
        if return_flights_2D.shape[0] > 0:
            return_flights_sorted_2D = return_flights_2D.sort_values(by = ['SCH_DEP_DTMZ'])
            return_flight_1D = return_flights_sorted_2D.iloc[0]       # Selecting return flight that is closest in time to outbound flight 1.
            arr_delay_1D.append(return_flight_1D['ARR_DELAY_MINUTES'])
            dep_delay_1D.append(return_flight_1D['DEP_DELAY_MINUTES'])
            flt_num_1D.append(return_flight_1D['FLT_NUM'])
            flt_duration_1D.append(return_flight_1D['FLT_ACTUAL_HR'])
            bank_code_1D.append(return_flight_1D['BANK_CD'])
            dep_date_Z_1D.append(return_flight_1D['SCH_DEP_DTZ'])
            dep_time_Z_1D.append(return_flight_1D['SCH_DEP_TMZ'])
            arr_date_Z_1D.append(return_flight_1D['SCH_ARR_DTZ'])
            arr_time_Z_1D.append(return_flight_1D['SCH_ARR_TMZ'])            
        else:
            arr_delay_1D.append(-10000)
            dep_delay_1D.append(-10000)
            flt_duration_1D.append(-10000)
            flt_num_1D.append(-10000)
            bank_code_1D.append('0')
            dep_date_Z_1D.append('0')
            dep_time_Z_1D.append('0')
            arr_date_Z_1D.append('0')
            arr_time_Z_1D.append('0')
            print('Error: Return flight not found in file FSU_fully_cleaned.csv.')
    else:
        arr_delay_1D.append(-10000)
        dep_delay_1D.append(-10000)
        flt_duration_1D.append(-10000)
        flt_num_1D.append(-10000)
        bank_code_1D.append('0')
        dep_date_Z_1D.append('0')
        dep_time_Z_1D.append('0')
        arr_date_Z_1D.append('0')
        arr_time_Z_1D.append('0')
        print('Error: Aircraft tail of outbound / return flight is not found in file FSU_fully_cleaned.csv.')

    return n_feeders, OD_1D, tail_1D, dep_delay_1D, arr_delay_1D, flt_num_1D, flt_duration_1D, bank_code_1D, dep_date_Z_1D, dep_time_Z_1D, arr_date_Z_1D, arr_time_Z_1D, outbound_1_SCH_ARR_DTMZ, return_flight_1D

In [8]:
def get_tooltips_3 (outbound_1_flight_ID):

    n_feeders, OD_1D, tail_1D, dep_delay_1D, arr_delay_1D, flt_num_1D, flt_duration_1D, bank_code_1D, dep_date_Z_1D, dep_time_Z_1D, arr_date_Z_1D, arr_time_Z_1D, outbound_1_SCH_ARR_DTMZ, return_flight_1D = get_tooltips_2 (outbound_1_flight_ID)
    
    return_flight_dep_dtz = return_flight_1D['SCH_DEP_DTZ']
    return_flight_OD = return_flight_1D['OD']
    return_flight_dep_tmz = return_flight_1D['SCH_DEP_TMZ']
    return_flight_num = return_flight_1D['FLT_NUM']
    return_flight_ID = return_flight_dep_dtz + return_flight_OD + return_flight_dep_tmz + str(return_flight_num)
    return_flight_ID = return_flight_ID.replace("-", "/")
    print('return_flight_ID =', return_flight_ID)
    outbound_2_flights_2D = feeders_2D[feeders_2D['id_f'] == return_flight_ID]

    if outbound_2_flights_2D.shape[0] > 0:
        outbound_2_dep_dtz_1D = outbound_2_flights_2D['dep_dtz_nf']
        outbound_2_OD_1D = outbound_2_flights_2D['od_nf']
        outbound_2_dep_tmz_1D = outbound_2_flights_2D['dep_tmz_nf']
        outbound_2_flt_num_1D = outbound_2_flights_2D['flt_num_nf']
        outbound_2_flights_ID_1D = outbound_2_dep_dtz_1D + outbound_2_OD_1D + outbound_2_dep_tmz_1D + str(outbound_2_flt_num_1D)
        outbound_2_flights_ID_1D = outbound_2_flights_ID_1D.replace("-", "/")
    else:
        print('Error: Return flight ' + return_flight_ID + ' not found in file bookings_ids+pax_date.csv. \n')

    n_outbound_2 = outbound_2_flights_2D.shape[0]

    for x in range(n_outbound_2):
        d = FSU_2D[(FSU_2D['SCH_DEP_DTZ'] == outbound_2_flights_2D['dep_dtz_nf'].iloc[x]) & (FSU_2D['OD'] == outbound_2_flights_2D['od_nf'].iloc[x]) & (FSU_2D['SCH_DEP_TMZ'] == outbound_2_flights_2D['dep_tmz_nf'].iloc[x]) & (FSU_2D['FLT_NUM'] == outbound_2_flights_2D['flt_num_nf'].iloc[x])]
        if d.shape[0] != 0:
            OD_1D.extend(d['OD'])
            arr_delay_1D.extend(d['ARR_DELAY_MINUTES'])
            dep_delay_1D.extend(d['DEP_DELAY_MINUTES'])
            flt_num_1D.extend(d['FLT_NUM'])
            flt_duration_1D.extend(d['FLT_ACTUAL_HR'])
            tail_1D.extend(d['TAIL'])
            bank_code_1D.extend(d['BANK_CD'])
            dep_date_Z_1D.extend(d['SCH_DEP_DTZ'])
            dep_time_Z_1D.extend(d['SCH_DEP_TMZ'])
            arr_date_Z_1D.extend(d['SCH_ARR_DTZ'])
            arr_time_Z_1D.extend(d['SCH_ARR_TMZ'])
        else:
            arr_delay_1D.append(-10000)
            dep_delay_1D.append(-10000)
            flt_duration_1D.append(-10000)
            flt_num_1D.append(-10000)
            OD_1D.extend('0')
            tail_1D.extend('0')
            bank_code_1D.append('0')
            dep_date_Z_1D.append('0')
            dep_time_Z_1D.append('0')
            arr_date_Z_1D.append('0')
            arr_time_Z_1D.append('0')

    tooltips_2D = []
    for i in range(len(OD_1D)):
        tooltips_2D.append(" Origin + destination code = " + OD_1D[i] + "<br> Aircraft tail = " + tail_1D[i] + " <br> Departure delay (minutes) = " + str(dep_delay_1D[i]) + "<br> Arrival delay (minutes) = " + str(arr_delay_1D[i]) + "<br> Flight number = " + str(int(flt_num_1D[i])) + "<br> Flight duration (hours) = " + str(flt_duration_1D[i]) + "<br> PTY bank code = " + bank_code_1D[i] + "<br> Scheduled departure date (Zulu) = " + dep_date_Z_1D[i] + "<br> Scheduled departure time (Zulu) = " + dep_time_Z_1D[i] + "<br> Scheduled arrival date (Zulu) = " + arr_date_Z_1D[i] + "<br> Scheduled arrival time (Zulu) = " + arr_time_Z_1D[i])

    return n_feeders, n_outbound_2, arr_delay_1D, dep_delay_1D, OD_1D, tooltips_2D

In [9]:
delay_category_1D = ['No arrival delay', '1 to 15', '16 to 30', '31 to 45', '46 to 60', 'More than 60', 'Data not available']

delay_colors_1D = ['rgb(100,100,100)', 'rgb(0,255,255)', 'rgb(0,255,0)', 'rgb(255,255,0)', 'rgb(255,127,0)', 'rgb(255,0,0)', 'rgb(230,230,230)']

def get_traces_data(outbound_1_flight_ID): #, tooltips_2D):
    
    position = {}

    n_feeders, n_outbound_2, arr_delay_1D, dep_delay_1D, OD_1D, tooltips_2D = get_tooltips_3(outbound_1_flight_ID)
    
    G = Graph()
    G.add_vertices(n_feeders + n_outbound_2 + 2)


    for x in range(0, n_feeders):
        G.add_edges([(x, n_feeders)])
        position[x] = [(2 * x) - n_feeders, 0]

    G.add_edges([(n_feeders, n_feeders + 1)])
    position[n_feeders] = [0, 1]
    position[n_feeders + 1] = [0, 2]

    for n, x in enumerate(range(n_feeders + 2, n_feeders + 2 + n_outbound_2)):
        G.add_edges([(n_feeders + 1, x)])
        position[x] = [(2 * n) - n_outbound_2, 3]

    # print('\n position = \n', position, '.')

    Y = [position[k][1] for k in range(n_feeders + n_outbound_2 + 2)]
    # print('\n Y = \n', Y, '.')

    M = max(Y)
    # print('\n M =', M, '.')

    es = EdgeSeq(G) # sequence of edges
    E = [e.tuple for e in G.es] # list of edges

    Xn = []
    Yn = []
    
    Xn = [position[k][0] for k in range(len(position))]
    # print('\n Xn = \n', Xn, '.')

    Yn = [M-position[k][1]+1 for k in range(len(position))]
    # print('\n Yn = \n', Yn, '.')

    Xe = []
    Ye = []

    for edge in E:
        Xe+=[position[edge[0]][0], position[edge[1]][0], None]
        Ye+=[M-position[edge[0]][1]+1, M-position[edge[1]][1]+1, None]

    # print('\n Xe = \n', Xe, '.')
    # print('\n Ye = \n', Ye, '.')
    
    colors_1D = []

    delays_minutes_1D = arr_delay_1D
    
    for x in range(n_feeders + n_outbound_2 + 2):
        if (delays_minutes_1D[x] > 0 and delays_minutes_1D[x] <= 15):
            c = delay_colors_1D[1]
        elif (delays_minutes_1D[x] > 15 and delays_minutes_1D[x] <= 30):
            c = delay_colors_1D[2]
        elif (delays_minutes_1D[x] > 30 and delays_minutes_1D[x] <= 45):
            c = delay_colors_1D[3]
        elif (delays_minutes_1D[x] > 45 and delays_minutes_1D[x] <= 60):
            c = delay_colors_1D[4]
        elif (delays_minutes_1D[x] > 60):
            c = delay_colors_1D[5]
        elif delays_minutes_1D[x] == -10000:     # Data not available
            c = delay_colors_1D[6] 
        else:                                           # No delay.
            c = delay_colors_1D[0]
        colors_1D.append(c)

    return Xe, Ye, Xn, Yn, colors_1D, tooltips_2D, OD_1D # trace_edges, trace_flight_nodes #, traces_legend_nodes_1D

# Create widget(s)

In [10]:
from ipywidgets import widgets

# years_1D = np.arange(2018, 2022, 1)

# year = widgets.Dropdown(
#     options = years_1D,
#     value = 2019,
#     description = 'Year:')

# months_1D = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

# month = widgets.Dropdown(
#     options = months_1D,
#     value = "October",
#     description = 'Month:')

# days_1D = np.arange(1, 32, 1)

# day = widgets.Dropdown(
#     options = days_1D,
#     value = 1,
#     description = 'Day:')
   
# destination = widgets.Dropdown(
#     options = destinations_unique_1D,
#     value = 'HAV',
#     description = 'Destination airport:')

# hours_1D = np.arange(0, 24, 1)

# hour = widgets.Dropdown(
#     options = hours_1D,
#     value = 12,
#     description='Hour:')

# minutes_1D = np.arange(0, 60, 1)
# minute = widgets.Dropdown(
#     options = minutes_1D,
#     value = 32,
#     description = 'Minute:')

# flight_numbers_1D = np.arange(100, 1000, 1)
# flight_number = widgets.Dropdown(
#     options = flight_numbers_1D,
#     value = 371,
#     description = 'Flight number:')

# flight_number = widgets.IntSlider(
#     value = 371,
#     min = 1,
#     max = 999,
#     step = 1,
#     description = 'Flight number:',
#     continuous_update = False)

flight_ID_widget = widgets.Dropdown(
    options = flight_ID_unique_1D,
    value = '2019/10/01PTYSDQ17:43128',
    description = 'Flight ID:')

# Create Plotly traces

In [11]:
# Xe, Ye, Xn, Yn, colors_1D, tooltips_2D, delay_category_1D, OD_1D = get_traces_data(flight_ID_widget.value) # , tooltips_2D)

def create_traces(outbound_1_flight_ID):
    traces_1D = []

    Xe, Ye, Xn, Yn, colors_1D, tooltips_2D, OD_1D = get_traces_data(outbound_1_flight_ID)
    
    # Appending trace of edges:
    traces_1D.append( go.Scatter(x = Xe,
                           y = Ye,
                           mode = 'lines',
                           name = 'Connecting passengers',
                           line = dict(color = 'rgb(200,200,200)', width=5), 
                           #text = pax_transfer_PTY_1D, 
                           hoverinfo = 'none'
                           ))

    # Appending trace of flight nodes:
    traces_1D.append( go.Scatter(x = Xn,
                          y = Yn,
                          mode = 'markers',
                          name = '',
                          marker = dict(symbol = 'circle',
                                        size = 40,
                                        color = colors_1D,    #'#DB4551',
                                        line = dict(color='rgb(50,50,50)', width=1)
                                        ),
                          text = tooltips_2D,
                          hoverinfo = 'text',
                          opacity = 0.8
                          ) )

    # Appending traces of legend nodes:
    for i in range(len(delay_category_1D)):
        traces_1D.append( go.Scatter(x = [i],
                      y = [min(Yn) - 1],
                      mode = 'markers',
                      name = delay_category_1D[i],
                      marker = dict(symbol = 'circle',
                                    size = 40,
                                    color = delay_colors_1D[i],    #'#DB4551',
                                    line = dict(color='rgb(50,50,50)', width=1)
                                    ),
                      hoverinfo = 'none',
                      opacity = 0.8
                      ) )
    
    return traces_1D

In [12]:
traces_1D = create_traces(flight_ID_widget.value)
# Xe, Ye, Xn, Yn, colors_1D, tooltips_2D, OD_1D = get_traces_data(flight_ID_widget.value)

fig = go.FigureWidget(data = traces_1D,
                        layout = go.Layout(
                            title = dict(
                                text = 'Copa Airlines flight network for selected outbound flight.'
                            )
                            # barmode='overlay'
                        ))

Error: Feeder data not found.
return_flight_ID = 2019/10/01SDQPTY21:40307


# Create OD text inside the node circles via annotations

In [13]:
def make_annotations(Xn, Yn, OD_1D, font_size = 12, font_color='rgb(0,0,0)'):
    print('len(Xn) = ' + str(len(Xn)) + ', \t len(Yn) = ' + str(len(Yn)) + ', \t len(OD_1D) = ' + str(len(OD_1D)) + '.')
    
    if len(Xn) != len(Yn):
        raise ValueError('The lists Xn and Yn must have the same length.')
    
    annotations = []
    for k in range(len(OD_1D)):
        annotations.append(
            dict(
                text = OD_1D[k], # or replace labels with a different list for the text within the circle
                x = Xn[k], y = Yn[k], # x = pos[k][0], y = 2*M-position[k][1],
                xref = 'x1', yref = 'y1',
                font = dict(color = font_color, size = font_size),
                showarrow = False)
        )
    return annotations

# Add axis specifications, create the layout, and save image as PNG file

In [14]:
# axis = dict(showline = True, # hide axis line, grid, tick-labels and  title
#             zeroline = False,
#             showgrid = True,
#             showticklabels = True,
#             )

# fig.update_layout(title = 'Copa Airlines flight network',
#               annotations = make_annotations(Xn, Yn, OD_1D),
#               font_size = 15,
#               showlegend = True,
#               xaxis = axis,
#               yaxis = axis,
#               margin = dict(l = 40, r = 40, b = 85, t = 100),
#               hovermode = 'closest',
#               plot_bgcolor = 'rgb(255,255,255)'
#               )

# fig.show()    # Display graph.

# Save as PNG image:
    # fig.write_image("../images/Plotly with widget - Copa flight tree with mutiple feeders, for outbound flight " + 
                    # outbound_1_dep_dtz + '_' + outbound_1_OD + '_' + time_reformat + '_' + 
                    # outbound_1_flt_num_str + ").png", width = 1500, height = 1000)

In [15]:
# def validate():
    
#     if flight_ID_widget.value in feeders_2D['id_nf'].unique():
#         return True
#     else:
#         return False

def response(change):
    print('============================================================================== \n')
    Xe, Ye, Xn, Yn, colors_1D, tooltips_2D, OD_1D = get_traces_data(flight_ID_widget.value) # , tooltips_2D)
    with fig.batch_update():
        fig.data[0].x = Xe
        fig.data[0].y = Ye
        fig.data[1].x = Xn
        fig.data[1].y = Yn
        fig.data[1].marker['color'] = colors_1D
        fig.data[1].text = tooltips_2D
        fig.update_layout( annotations = make_annotations(Xn, Yn, OD_1D) )

flight_ID_widget.observe(response, names = "value")

# origin = 'PTY'

In [16]:
Xe, Ye, Xn, Yn, colors_1D, tooltips_2D, OD_1D = get_traces_data(flight_ID_widget.value) # , tooltips_2D)

fig.update_layout( annotations = make_annotations(Xn, Yn, OD_1D) )

widgets.VBox([widgets.HBox([flight_ID_widget]), fig])

Error: Feeder data not found.
return_flight_ID = 2019/10/01SDQPTY21:40307
len(Xn) = 42, 	 len(Yn) = 42, 	 len(OD_1D) = 42.



Error: Feeder data not found.
return_flight_ID = 2019/10/02HAVPTY10:35379
Error: Return flight 2019/10/02HAVPTY10:35379 not found in file bookings_ids+pax_date.csv. 

len(Xn) = 21, 	 len(Yn) = 21, 	 len(OD_1D) = 21.

return_flight_ID = 2019/10/03HAVPTY10:20295
Error: Return flight 2019/10/03HAVPTY10:20295 not found in file bookings_ids+pax_date.csv. 

len(Xn) = 3, 	 len(Yn) = 3, 	 len(OD_1D) = 3.
